# Schwarzschild black-hole immersed in an electric field

The action of Entanled Relativiy reads as follows

<!-- $S = \int \mathrm{d}^4 x\sqrt{-g} \left( \frac{\Phi R}{2\kappa} + \sqrt{\Phi} \mathcal{L}_m \right)$ -->
$S = -\frac{1}{2}\int \mathrm{d}^4 x\sqrt{-g} \frac{\mathcal{L}_m^2}{R}$

The fields equations are

<!-- $G_{\mu\nu} = \frac{\kappa}{\sqrt{\Phi}}T_{\mu\nu} + \frac{1}{\Phi} \left( \nabla_\mu \nabla_\nu - g_{\mu\nu} \Box \right) \Phi$ -->
$G_{\mu\nu} = -\frac{R}{\mathcal{L}_m}T_{\mu\nu} + \frac{R^2}{\mathcal{L}_m^2} \left( \nabla_\mu \nabla_\nu - g_{\mu\nu} \Box \right) \frac{\mathcal{L}_m^2}{R^2}$,

and $\nabla_\sigma \left(\frac{\mathcal{L}}{R} F^{\mu \sigma}\right) = 0$,

with $\mathcal{L}_m = -\frac{1}{2} F_{\mu\nu}F^{\mu\nu}$.

The solution for the metric is
$$ds^2 = \Lambda^{\frac{28}{13}} \left[\left(1 - \frac{r_S}{r}\right)^{-1}dr^2 + r^2 d\theta^2 - \left(1 - \frac{r_S}{r}\right) dt^2 \right] + \Lambda^{-\frac{20}{13}} r^2 sin^2(\theta) d\varphi^2,$$
with
$$\Lambda = 1 + \frac{13}{48} E^2 r^2 sin^2(\theta).$$

The solution for the electromagnetic potential 4-vector is
$$A_t = E \left(1-\frac{r_S}{r} \right) r \cos \theta.$$

The solution is such that
$$-\frac{\mathcal{L}_m}{R} = \Lambda^{-2/13}.$$

Below, we verify that solution.

In [1]:
version()

'SageMath version 9.5, Release Date: 2022-01-30'

In [2]:
%display latex

In [3]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import laplacian
from sage.manifolds.operators import grad

In [4]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:\varphi")

## Definition of the fields

In [6]:
g = M.metric()
rs = var('r_S')
assume(rs>=0)
assume(rs<r) # focus on exterior solution
B = var('E')

Lambda(r,th) = 1 + 13 / 48 * B**2 * (r*sin(th))**2

g[0,0] = -(1- rs/r) * Lambda(r,th)**(28 / 13)
g[1,1] = 1/(1- rs/r) * Lambda(r,th)**(28 / 13)
g[2,2] = r**2 * Lambda(r,th)**(28 / 13)
g[3,3] = (r*sin(th))**2 / Lambda(r,th)**(20 / 13)

pot_vec = M.tensor_field(0,1,name='A')
pot_vec[0]= B * (1 - rs / r) * r * cos(th)
pot_vec[1]=0
pot_vec[2]=0
pot_vec[3]= 0

In [7]:
g.display()

g = (13/48*E^2*r^2*sin(th)^2 + 1)^(28/13)*(r_S/r - 1) dt⊗dt - (13/48*E^2*r^2*sin(th)^2 + 1)^(28/13)/(r_S/r - 1) dr⊗dr + (13/48*E^2*r^2*sin(th)^2 + 1)^(28/13)*r^2 dth⊗dth + r^2*sin(th)^2/(13/48*E^2*r^2*sin(th)^2 + 1)^(20/13) dph⊗dph

In [8]:
pot_vec.display()

A = -E*r*(r_S/r - 1)*cos(th) dt

In [9]:
nab = g.connection() 

In [10]:
Ric = g.ricci() #Ricci tensor
R = g.ricci_scalar() #Ricci scalar
G = Ric-R*g/2 #Einstein tensor

## Electromagnetic field equation

In [11]:
DF = nab(pot_vec) 

In [12]:
F=(DF['_ij']-DF['_ji'])._tensor
Fuu = F.up(g)
F.display()

E*cos(th) dt⊗dr - (E*r - E*r_S)*sin(th) dt⊗dth - E*cos(th) dr⊗dt + (E*r - E*r_S)*sin(th) dth⊗dt

In [13]:
Lm = -F['_ij']*F.up(g)['^ij']/2
# Lm.display()
Lm.expr().canonicalize_radical().factor()

-10616832*3^(4/13)*2^(3/13)*(r_S*sin(th)^2 - r)*E^2/((13*E^2*r^2*sin(th)^2 + 48)^(56/13)*r)

In [14]:
(221184*3**(5/13)*2**(7/13)/48**(44/13)).canonicalize_radical()

1

In [15]:
"""Maxwell's equation"""

N = nab(Lm / R *F.up(g))['_i^ij']
N == 0

True

## Metric field equation

In [16]:
"""Einstein's equation"""


T = 2*(F.up(g,1)['_i^j']*F['_kj'] + g*Lm/2) #Le tenseur énergie impuslion

S = (nab(nab(Lm**2 / R**2 )) - g*(Lm**2 / R**2).dalembertian()) * R**2 / Lm**2 #Le tenseur en plus dans l'équation

G == - R / Lm * T + S

True

## Check scalar-field form

In [17]:
"Test that Lambda**(-2/13) + Lm/R == 0"
Lambda**(-2/13) + Lm/R == 0

True

## Properties

In [18]:
# Lm.expr()
Lm.expr().canonicalize_radical().factor().factor()

-10616832*3^(4/13)*2^(3/13)*(r_S*sin(th)^2 - r)*E^2/((13*E^2*r^2*sin(th)^2 + 48)^(56/13)*r)

In [19]:
(10616832 * 3**(4/13) * 2**(3/13) / 48**(56/13)).canonicalize_radical()

1

$$\Rightarrow \mathcal{L}_m = E^2 \Lambda^{-56/13}\left(1- \frac{r_S~\sin^2 \theta}{r}\right)$$

In [20]:
# R.expr()
R.expr().canonicalize_radical().factor().factor()

5308416*3^(2/13)*2^(8/13)*(r_S*sin(th)^2 - r)*E^2/((13*E^2*r^2*sin(th)^2 + 48)^(54/13)*r)

In [21]:
(5308416*3**(2/13)*2**(8/13) / 48**(54/13)).canonicalize_radical() 

1

$$\Rightarrow R = -E^2 \Lambda^{-54/13}\left(1- \frac{r_S~\sin^2 \theta}{r}\right)$$

### Petrov classification

In [22]:
C = (g.weyl()).down(g)

In [23]:
"""null tetrad definition"""

K = M.one_form({X.frame(): [sqrt(-g[0,0]/2), sqrt(g[1,1]/2), 0, 0]})
L = M.one_form({X.frame(): [sqrt(-g[0,0]/2), -sqrt(g[1,1]/2), 0, 0]})
Mb = M.one_form({X.frame(): [0, 0, sqrt(g[2,2]/2), i*sqrt(g[3,3]/2)]})
MB = M.one_form({X.frame(): [0, 0, sqrt(g[2,2]/2), -i*sqrt(g[3,3]/2)]})
k = K.up(g)
l = L.up(g)
m = Mb.up(g)
mb = MB.up(g)

### Verification of null tetrad

In [24]:
g(k,k).expr()

0

In [25]:
g(l,l).expr()

0

In [26]:
g(m,m).expr()

0

In [27]:
g(mb,mb).expr()

0

In [28]:
g(k,l).expr()

-1

In [29]:
g(m,mb).expr()

1

### Computation of the Weyl scalars

In [30]:
P00 = k*m*k*m
P0 = C['_{abcd}']*P00['^{abcd}']
P0.expr().canonicalize_radical().factor()

27648*3^(2/13)*2^(8/13)*(35*E^2*r^2*sin(th)^2 - 144)*E^2*(r - r_S)*sin(th)^2/((13*E^2*r^2*sin(th)^2 + 48)^(54/13)*r)

In [31]:
P11 = k*l*k*m
P1 = C['_{abcd}']*P11['^{abcd}']
P1.expr().canonicalize_radical().factor()

27648*3^(2/13)*2^(8/13)*(35*E^2*r^2*sin(th)^2 - 144)*E^2*sqrt(r - r_S)*cos(th)*sin(th)/((13*E^2*r^2*sin(th)^2 + 48)^(54/13)*sqrt(r))

In [32]:
P22 = k*m*mb*l
P2 = C['_{abcd}']*P22['^{abcd}']
P2.expr().canonicalize_radical().factor()

-1152*3^(2/13)*2^(8/13)*(840*E^4*r^5*sin(th)^4 - 423*E^4*r^4*r_S*sin(th)^4 - 560*E^4*r^5*sin(th)^2 - 3456*E^2*r^3*sin(th)^2 + 1248*E^2*r^2*r_S*sin(th)^2 + 2304*E^2*r^3 + 2304*r_S)/((13*E^2*r^2*sin(th)^2 + 48)^(54/13)*r^3)

In [33]:
P33 = l*k*l*mb
P3 = C['_{abcd}']*P33['^{abcd}']
P3.expr().canonicalize_radical().factor()

-27648*3^(2/13)*2^(8/13)*(35*E^2*r^2*sin(th)^2 - 144)*E^2*sqrt(r - r_S)*cos(th)*sin(th)/((13*E^2*r^2*sin(th)^2 + 48)^(54/13)*sqrt(r))

In [34]:
P44 = l*mb*l*mb
P4 = C['_{abcd}']*P44['^{abcd}']
P4.expr().canonicalize_radical().factor()

27648*3^(2/13)*2^(8/13)*(35*E^2*r^2*sin(th)^2 - 144)*E^2*(r - r_S)*sin(th)^2/((13*E^2*r^2*sin(th)^2 + 48)^(54/13)*r)